<a href="https://colab.research.google.com/github/sanneen/neural-network-challenge-2/blob/main/Another_copy_of_attrition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 1: Preprocessing

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [ ]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [ ]:
# Create y_df with the Attrition and Department columns
y_df=attrition_df[['Attrition','Department']]
y_df



,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development
...,...,...
1465,No,Research & Development
1466,No,Research & Development
1467,No,Research & Development
1468,No,Sales


In [ ]:
# Create a list of at least 10 column names to use as X data
X=attrition_df.drop(['Attrition','Department','BusinessTravel','EducationField','JobRole','MaritalStatus'],axis=1)

X.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   DistanceFromHome          1470 non-null   int64 
 2   Education                 1470 non-null   int64 
 3   EnvironmentSatisfaction   1470 non-null   int64 
 4   HourlyRate                1470 non-null   int64 
 5   JobInvolvement            1470 non-null   int64 
 6   JobLevel                  1470 non-null   int64 
 7   JobSatisfaction           1470 non-null   int64 
 8   NumCompaniesWorked        1470 non-null   int64 
 9   OverTime                  1470 non-null   object
 10  PercentSalaryHike         1470 non-null   int64 
 11  PerformanceRating         1470 non-null   int64 
 12  RelationshipSatisfaction  1470 non-null   int64 
 13  StockOptionLevel          1470 non-null   int64 
 14  TotalWorkingYears       

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X['OverTime']=le.fit_transform(X['OverTime'])
X['OverTime'].value_counts()


,count
OverTime,
0,1054
1,416


In [ ]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y_df,random_state=2,test_size=0.2)


In [ ]:
# Create a StandardScaler
sc=StandardScaler()

# Fit the StandardScaler to the training data
sc.fit(X_train)

# Scale the training and testing data

X_train_scaled=sc.transform(X_train)
X_test_scaled=sc.transform(X_test)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
deptencoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
deptencoder.fit(y_train['Department'].values.reshape(-1,1))

# Create two new variables by applying the encoder
# to the training and testing data
y_dept_train=deptencoder.transform(y_train['Department'].values.reshape(-1,1))
y_dept_test=deptencoder.transform(y_test['Department'].values.reshape(-1,1))
print(y_dept_train)




[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]


In [ ]:
# Create a OneHotEncoder for the Attrition column
attencoder=OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attencoder.fit(y_train['Attrition'].values.reshape(-1,1))

# Create two new variables by applying the encoder
# to the training and testing data
y_att_train=attencoder.transform(y_train['Attrition'].values.reshape(-1,1))
y_att_test=attencoder.transform(y_test['Attrition'].values.reshape(-1,1))

y_att_test

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.

In [ ]:
# Find the number of columns in the X training data
len(X_train.columns)


21

## Create, Compile, and Train the Model

In [ ]:

from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Input, Dense
# Create the input layer
input_layer = Input(shape=(len(X_train.columns),), name='input_layer')

# Create at least two shared layers
dense1=Dense(64,activation='relu')(input_layer)
dense2=Dense(64,activation='relu')(dense1)

In [ ]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
dept_dense=Dense(64,activation='relu')(dense2)

# Create the output layer
dept_output=Dense(3,activation='softmax',name='dept')(dept_dense)


In [ ]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
att_dense=Dense(64,activation='relu')(dense2)

# Create the output layer
att_output=Dense(2,activation='softmax',name='att')(att_dense)


In [ ]:
# Create the model
model=Model(inputs=input_layer,outputs=[dept_output,att_output])

# Compile the model
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics={'dept':'accuracy','att':'accuracy'})

# Summarize the model
model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 21)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_12 (Dense)          │ (None, 64)             │          1,408 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_13 (Dense)          │ (None, 64)             │          4,160 │ dense_12[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_16 (Dense)          │ (None, 64)             │          4,160 │ dense_13[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_17 (Dense)          │ (None, 64)             │          4,160 │ dense_13[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dept (Dense)              │ (None, 3)              │            195 │ dense_16[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ att (Dense)               │ (None, 2)              │            130 │ dense_17[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 14,213 (55.52 KB)

 Trainable params: 14,213 (55.52 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the model
model.fit(X_train_scaled,[y_dept_train,y_att_train], epochs=100)



Epoch 1/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - att_accuracy: 0.6703 - att_loss: 0.5904 - dept_accuracy: 0.4464 - dept_loss: 1.1069 - loss: 1.6974
Epoch 2/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - att_accuracy: 0.8556 - att_loss: 0.3831 - dept_accuracy: 0.6411 - dept_loss: 0.7990 - loss: 1.1822
Epoch 3/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - att_accuracy: 0.8664 - att_loss: 0.3615 - dept_accuracy: 0.6374 - dept_loss: 0.7906 - loss: 1.1521
Epoch 4/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - att_accuracy: 0.8676 - att_loss: 0.3370 - dept_accuracy: 0.6630 - dept_loss: 0.7318 - loss: 1.0688
Epoch 5/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - att_accuracy: 0.8669 - att_loss: 0.3128 - dept_accuracy: 0.6390 - dept_loss: 0.7473 - loss: 1.0602
Epoch 6/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - att_accuracy: 0.8878 - att_loss: 0.3081 - dept_accuracy: 0.6527 - dept_loss: 0.7258 - loss: 1.0340
Epoch 7/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - att_accuracy: 0.8655 - att_loss: 0.

In [ ]:
# Evaluate the model with the testing data
model.evaluate(X_test_scaled,[y_dept_test,y_att_test])

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - att_accuracy: 0.8593 - att_loss: 1.8030 - dept_accuracy: 0.6754 - dept_loss: 2.7070 - loss: 4.5758  


[4.718738555908203,
 2.705923557281494,
 1.6502866744995117,
 0.8469387888908386,
 0.6428571343421936]

In [ ]:
from sklearn.metrics import accuracy_score
prediction=model.predict(X_test_scaled)
prediction


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


[array([[1.85226323e-08, 9.85895574e-01, 1.41043868e-02],
        [1.23930983e-02, 8.99474323e-03, 9.78612185e-01],
        [1.26169283e-07, 9.99999821e-01, 3.23320783e-08],
        [9.06362008e-09, 3.26358855e-01, 6.73641145e-01],
        [6.06119029e-05, 9.99865770e-01, 7.36004222e-05],
        [1.06847845e-04, 9.99890506e-01, 2.64590949e-06],
        [3.34898941e-02, 9.13835684e-05, 9.66418743e-01],
        [7.14034911e-08, 3.16938567e-05, 9.99968231e-01],
        [1.37655761e-07, 9.99999821e-01, 1.83361226e-09],
        [1.15863645e-10, 9.99999702e-01, 2.16356852e-07],
        [2.22239265e-04, 9.99774456e-01, 3.21855737e-06],
        [4.46141377e-04, 9.99553859e-01, 1.04002015e-11],
        [3.64551641e-04, 8.63602579e-01, 1.36032864e-01],
        [1.43594280e-01, 8.27673018e-01, 2.87325773e-02],
        [8.50355718e-04, 9.99128103e-01, 2.14179672e-05],
        [9.49764489e-10, 9.99999940e-01, 3.87586319e-08],
        [1.93657593e-06, 2.20812399e-06, 9.99995768e-01],
        [9.691

In [ ]:
# Calculate accuracy for department prediction
dept_accuracy = accuracy_score(y_dept_test,dept_pred)


In [ ]:
# Calculate accuracy for attrition prediction
att_accuracy = accuracy_score(y_att_test, att_pred)

In [ ]:
# Print the accuracy for both department and attrition

print(f'Department predictions accuracy:{accuracy_score(y_dept_test,dept_pred)}')
print(f'Attrition predictions accuracy:{accuracy_score(y_att_test,att_pred)}')

Department predictions accuracy:0.6428571428571429
Attrition predictions accuracy:0.8469387755102041


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1.
2.
3.

1- accuracy is seldom a best metric when X vector is large
2- used softmax, too many department types
3- more data